In [7]:
from tqdm import tqdm

In [8]:
from libs.feature_extraction.vectorizers import BertVectorizer
import numpy as np

bert = BertVectorizer("microsoft/unixcoder-base")
vectors = bert.load_vectors("data/comment_finder/vectors/bert_vectors.npy")
zero_vectors = np.zeros((vectors.shape[0], 4096))
zero_vectors[:vectors.shape[0], :vectors.shape[1]] = vectors
vectors = zero_vectors

Loading model(microsoft/unixcoder-base)...


In [10]:
import pandas as pd
from nltk.translate import bleu_score
from numpy.linalg import norm
import numpy as np

df = pd.read_csv("./data/comment_finder/all.csv")

chencherry = bleu_score.SmoothingFunction()

def getBleuScore(test_idx, match_idx):
    ref = df.iloc[[test_idx]].to_records()[0][2]
    hyp = df.iloc[[match_idx]].to_records()[0][2]
    bleu_score_val = bleu_score.sentence_bleu([ref], hyp, smoothing_function=chencherry.method1)
    return bleu_score_val


def find_similiar(data_vector, test_id):

    A = data_vector[test_id]

    max_cosine = 0
    max_sim_id=-1
    for i, B in enumerate(data_vector):
        cosine = np.dot(A,B)/(norm(A)*norm(B))
        if i == test_id:
            continue
        if cosine>max_cosine:
            max_sim_id = i
            max_cosine = cosine
    return max_sim_id, max_cosine

In [11]:
test_df = pd.read_csv("./results/bert_cosine_bleu_C1_F5/fold_1.csv")
test_series = test_df.test_idx

sum = 0
toplam = 1000 #len(test_series)
for index, test_idx in tqdm(enumerate(test_series), total=toplam):
    if index > toplam-1:
        break
    max_sim_id, max_cosine = find_similiar(vectors, test_idx)
    bleu_score_val = getBleuScore(test_idx,max_sim_id)
    sum = sum + bleu_score_val
    
avg_bleu_score = sum / toplam
display(avg_bleu_score)

100%|██████████| 100/100 [03:14<00:00,  1.95s/it]


0.1258159174792227